# Song Stream Predictions

### Connecting to Snowflake

This script sets up a connection to Snowflake using SQLAlchemy, facilitates query execution through context management, and provides specific functions (get_sodatone_query_dfs and get_sodatone_query_df) tailored for interacting with Snowflake in a Sodatone-specific context. Sodatone is a specialized data analytics platform tailored for the music industry. It provides comprehensive insights into streaming data from major platforms like Spotify and Apple Music. With Sodatone, we can track performance metrics, analyze audience demographics, and leverage predictive analytics to drive date-driven signings

In [3]:
# imports
import sys
import pandas as pd
from io import BytesIO
import numpy as np
sys.path.append('..')
import snowflake
import contextlib
from sqlalchemy import create_engine
from typing import List
from IPython.display import display
# import okta1

import os
from dotenv import load_dotenv
load_dotenv()



# import dodb
pd.options.display.max_columns = None

# WMG Snowflake general
SNOWFLAKE_AUTHENTICATOR="https://wmg.okta.com"
SNOWFLAKE_ACCOUNT="wmg-datalab"

# Sodatone-specific
SNOWFLAKE_SODATONE_ROLE="ENT_OKTA_SNOWFLAKE_DATALAB_ATLANTIC"
SNOWFLAKE_SODATONE_WAREHOUSE="ATLANTIC_SANDBOX_WH_XS"



# --- CONFIG ---
# your okta email
SNOWFLAKE_OKTA_USER = os.getenv("OKTA_EMAIL")
# your okta password
SNOWFLAKE_OKTA_PASSWORD = os.getenv("OKTA_PASSWORD")
def _get_engine(
        role: str,
        warehouse: str,
        user: str = SNOWFLAKE_OKTA_USER,
        password: str = SNOWFLAKE_OKTA_PASSWORD,
        account: str = SNOWFLAKE_ACCOUNT,
        authenticator: str = SNOWFLAKE_AUTHENTICATOR):
    return create_engine(
        _get_engine_url(role, warehouse, user, password, account),
        connect_args={
            # this will open your computer's default web-browser so you can auth with okta
            # then it will redirect to localhost with a token... that token is consumed by
            # the snowflake engine in order to authenticate queries
            # however, the engine needs to be recreated because of context decorator, so
            # sql queries should be grouped into an iterable of multiple statements to
            # prevent annoying window popups
            'authenticator': 'externalbrowser',
        },
    )

def _get_engine_url(role: str, warehouse: str, user: str, password: str, account: str) -> str:
    return (
        'snowflake://{user}:{password}@{account}/?warehouse={warehouse}&role={role}'.format(
            user=user,
            password=password,
            account=account,
            warehouse=warehouse,
            role=role,
        )
    )

@contextlib.contextmanager
def _get_conn(engine):
    conn = engine.connect()
    try:
        yield conn
    finally:
        conn.close()
        engine.dispose()

def _get_role_conn(role: str, warehouse: str):
    return _get_conn(_get_engine(role, warehouse))

def get_results_as_dfs(role: str, warehouse: str, sql_queries: List[str]) -> List[pd.DataFrame]:
    with _get_role_conn(role, warehouse) as conn:
        print(f"Running sql as {role} in warehouse: {warehouse}")
        return [pd.read_sql(x, conn) for x in sql_queries]

def get_sodatone_query_dfs(sql: List[str]):
    # sodatone specific
    return get_results_as_dfs(SNOWFLAKE_SODATONE_ROLE, SNOWFLAKE_SODATONE_WAREHOUSE, sql)
def get_results_as_df(role: str, warehouse: str, sql_query: str) -> pd.DataFrame:
    with _get_role_conn(role, warehouse) as conn:
        print(f"Running sql as {role} in warehouse: {warehouse}")
        return pd.read_sql(sql_query, conn)
    
def get_sodatone_query_df(sql: str):
    # sodatone specific
    return get_results_as_df(SNOWFLAKE_SODATONE_ROLE, SNOWFLAKE_SODATONE_WAREHOUSE, sql)

This query will be used to generate a DataFrame that we will use to train a one-week predictor (a model that predicts the number of streams one week in advance).

In [2]:
query1 = '''WITH joined AS (
  SELECT
    m.ARTIST,
    m.TITLE,
    m.UNIFIED_SONG_ID,
    m.DATE,
    t.RELEASE_DATE,
    m.THIS_DAY,
    m.THIS_WEEK,
    m.DATE - CAST(t.RELEASE_DATE AS DATE) AS DAYS_SINCE_RELEASE,
    (DAYOFWEEK(m.DATE) + 2) % 7 AS day_of_week,
    ROW_NUMBER() OVER(PARTITION BY m.UNIFIED_SONG_ID, m.DATE, t.RELEASE_DATE ORDER BY (SELECT NULL)) AS rn,
    END_OF_WEEK,
    POPULARITY
  FROM
    "SODATONE"."SODATONE"."LUMINATE_DAILY_SONG_METRICS" AS m
  JOIN
    "APP_REACT"."JOHN_S"."FINAL_TRACKS" AS t ON m.UNIFIED_SONG_ID = t.UNIFIED_SONG_ID
  WHERE
    m.METRIC_TYPE = 'Streaming On-Demand Audio'
    AND m.REGION = 'us'
    AND DAYS_SINCE_RELEASE >= 0
    AND DATE > '2024-01-01'
),
EarliestReleases AS (
    SELECT
        UNIFIED_SONG_ID,
        MIN(RELEASE_DATE) AS earliest_release_date
    FROM
        joined
    GROUP BY
        UNIFIED_SONG_ID
),
filtered_joined AS (
    SELECT
        j.*
    FROM
        joined AS j
    JOIN
        EarliestReleases e
    ON
        j.UNIFIED_SONG_ID = e.UNIFIED_SONG_ID
        AND j.RELEASE_DATE = e.earliest_release_date
    WHERE
        rn = 1
),
lag AS (
  SELECT *,
     THIS_DAY AS day_0,
     LAG(THIS_DAY, 1, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_1,
     LAG(THIS_DAY, 2, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_2,
     LAG(THIS_DAY, 3, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_3,
     LAG(THIS_DAY, 4, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_4,
     LAG(THIS_DAY, 5, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_5,
     LAG(THIS_DAY, 6, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_6,
     LEAD(THIS_WEEK, 8, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS target
  FROM filtered_joined
)
SELECT 
    * 
FROM 
    lag
WHERE 
    target IS NOT NULL
    AND END_OF_WEEK = TRUE
ORDER BY 
    UNIFIED_SONG_ID, DATE DESC;'''

This query will be used to generate a DataFrame that we will use to train a three-week predictor (a model that predicts the number of streams a song will have in three weeks).

In [1]:
query2 = '''WITH joined AS (
  SELECT
    m.ARTIST,
    m.TITLE,
    m.UNIFIED_SONG_ID,
    m.DATE,
    t.RELEASE_DATE,
    m.THIS_DAY,
    m.THIS_WEEK,
    m.DATE - CAST(t.RELEASE_DATE AS DATE) AS DAYS_SINCE_RELEASE,
    (DAYOFWEEK(m.DATE) + 2) % 7 AS day_of_week,
    ROW_NUMBER() OVER(PARTITION BY m.UNIFIED_SONG_ID, m.DATE, t.RELEASE_DATE ORDER BY (SELECT NULL)) AS rn,
    END_OF_WEEK,
    POPULARITY
  FROM
    "SODATONE"."SODATONE"."LUMINATE_DAILY_SONG_METRICS" AS m
  JOIN
    "APP_REACT"."JOHN_S"."FINAL_TRACKS" AS t ON m.UNIFIED_SONG_ID = t.UNIFIED_SONG_ID
  WHERE
    m.METRIC_TYPE = 'Streaming On-Demand Audio'
    AND m.REGION = 'us'
    AND DAYS_SINCE_RELEASE >= 0
),
EarliestReleases AS (
    SELECT
        UNIFIED_SONG_ID,
        MIN(RELEASE_DATE) AS earliest_release_date
    FROM
        joined
    GROUP BY
        UNIFIED_SONG_ID
),
filtered_joined AS (
    SELECT
        j.*
    FROM
        joined AS j
    JOIN
        EarliestReleases e
    ON
        j.UNIFIED_SONG_ID = e.UNIFIED_SONG_ID
        AND j.RELEASE_DATE = e.earliest_release_date
    WHERE
        rn = 1
),
lag AS (
  SELECT *,
     THIS_DAY AS day_0,
     LAG(THIS_DAY, 1, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_1,
     LAG(THIS_DAY, 2, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_2,
     LAG(THIS_DAY, 3, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_3,
     LAG(THIS_DAY, 4, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_4,
     LAG(THIS_DAY, 5, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_5,
     LAG(THIS_DAY, 6, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_6,
     LAG(THIS_DAY, 7, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_7,
     LAG(THIS_DAY, 8, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_8,
     LAG(THIS_DAY, 9, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_9,
     LAG(THIS_DAY, 10, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_10,
     LAG(THIS_DAY, 11, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_11,
     LAG(THIS_DAY, 12, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_12,
     LAG(THIS_DAY, 13, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_13,
     LAG(THIS_DAY, 14, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_14,
     LAG(THIS_DAY, 15, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_15,
     LAG(THIS_DAY, 16, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_16,
     LAG(THIS_DAY, 17, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_17,
     LAG(THIS_DAY, 18, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_18,
     LAG(THIS_DAY, 19, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_19,
     LAG(THIS_DAY, 20, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS day_20,
     LEAD(THIS_WEEK, 22, NULL) OVER(PARTITION BY UNIFIED_SONG_ID ORDER BY DATE ASC) AS target
  FROM filtered_joined
)
SELECT 
    *,
    day_20 + day_19 + day_18 + day_17 + day_18 + day_19 + day_20 AS naive
FROM 
    lag
WHERE 
    target IS NOT NULL
ORDER BY 
    UNIFIED_SONG_ID, DATE DESC;'''

In [3]:
df1 = get_sodatone_query_df(query1)
df1

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://wmg.okta.com/app/snowflake/exkab0aaa1IEBMlNX2p7/sso/saml?SAMLRequest=jZJBb%2BIwEIX%2FSuQ9J3ZCVViLUEGhEImyqEBV9eYkJnhx7KzHaei%2FXyeUFXto1Ztlvzfzjd8M706l9N64AaFVjMKAII%2BrTOdCFTHabR%2F8AfLAMpUzqRWP0TsHdDcaAitlRce1Pagn%2FqfmYD1XSAHtHmJUG0U1AwFUsZIDtRndjB%2BXNAoIrYy2OtMSXVm%2BdjAAbqwjvFhyEA7vYG1FMW6aJmh6gTYFjgghmPzETtVKflz0JzfTJ%2FoQk5tW7xROvv5gmwh1%2FoKvsNKzCOhiu13761%2BbLfLGF9R7raAuudlw8yYyvntangGgJSgLP2eWSZYGoHSzl%2BzIM11WtXX1AnfCe55jqQvhRk6mMaqOIk93k1592GXRqpin9Xp2ZM%2BD5HbV9Fe%2FZclfbvvNYrecFvP5YpYh7%2FmSadRmmgDUPFFtktZdkejGJ30%2FirYRob2QRv1gEPZekTd1SQrFbOe8wg300bIOjFUV%2FseM%2BenIUsIYC5PZ5FGuXqKqjwE0bkNF5z2hXXMz%2Bv70Q3zt%2B1i2lfv%2FZLrWUmTv3oM2JbOfxxMGYXcjcn%2FfSSkvmZDjPDccwMUkpW7uDWfW7bQ1NUd4dO76%2F1aP%2FgI%3D&RelayState=63854 to

,artist,title,unified_song_id,date,release_date,this_day,this_week,days_since_release,day_of_week,rn,end_of_week,popularity,day_0,day_1,day_2,day_3,day_4,day_5,day_6,target
0,Young Thug,Hercules,100070,2024-07-11,2016-02-05,15504,104046.0,3079,6,1,True,51,15504,15224.0,15367.0,14891.0,13279.0,14595.0,13974.0,100669
1,Young Thug,Hercules,100070,2024-07-04,2016-02-05,14410,104344.0,3072,6,1,True,51,14410,16173.0,15372.0,14658.0,12695.0,14821.0,15917.0,102646
2,Young Thug,Hercules,100070,2024-06-27,2016-02-05,15979,100850.0,3065,6,1,True,51,15979,15387.0,15061.0,14665.0,12847.0,14473.0,15932.0,103858
3,Young Thug,Hercules,100070,2024-06-20,2016-02-05,15343,103313.0,3058,6,1,True,51,15343,14816.0,14320.0,13438.0,12405.0,14523.0,16005.0,104343
4,Young Thug,Hercules,100070,2024-06-13,2016-02-05,15999,101589.0,3051,6,1,True,51,15999,15412.0,15053.0,13938.0,12305.0,14530.0,16076.0,100850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232615,Kylie Minogue,My Oh My (With Bebe Rexha & To,245207279,2024-07-11,2024-07-11,125551,0.0,0,6,1,True,67,125551,NaN,NaN,NaN,NaN,NaN,NaN,894109
232616,Tiesto & Lucas & Steve,ZENLESS,245214101,2024-07-11,2024-07-02,45169,61607.0,9,6,1,True,53,45169,47136.0,50775.0,62879.0,50048.0,59943.0,74695.0,195923
232617,Tiesto & Lucas & Steve,ZENLESS,245214101,2024-07-04,2024-07-02,27206,NaN,2,6,1,True,53,27206,15107.0,19281.0,NaN,NaN,NaN,NaN,389998
232618,Bizzy Banks,Back In The Mix,245773621,2024-07-11,2024-07-11,1238,NaN,0,6,1,True,35,1238,NaN,NaN,NaN,NaN,NaN,NaN,58350


In [4]:
df1.to_csv('../csv_files/snowflake_data.csv', index=False)

In [4]:
df2 = get_sodatone_query_df(query2)
df2

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://wmg.okta.com/app/snowflake/exkab0aaa1IEBMlNX2p7/sso/saml?SAMLRequest=jZJbb%2BIwEIX%2FSuR9TuyEIroWUNFStki0IC5Vd9%2BGxICLY2c9DoH%2B%2BnW4rNiHVvtm2ef4m5kz7bt9roKdsCiN7pA4YiQQOjWZ1OsOWcwH4S0J0IHOQBktOuQgkNx12wi5KnivdBs9Fb9LgS7wH2nk9UOHlFZzAyiRa8gFcpfyWe95xJOIcUAU1nkcOVsylJ61ca7glFZVFVWNyNg1TRhjlH2nXlVLvpErRPE1o7DGmdSoi2Xve%2FoEEVN2UyO8whMmZ%2BO91KcRfEVZnkTIn%2BbzSTgZz%2BYk6F26ezAay1zYmbA7mYrFdHQqAOsK8nWYgQMFywi1qVYKtiI1eVE6%2F1%2FkT3QlMqrMWvopDfsdUmxl1vx4bz4xPZj09%2B%2FJtPVY7N4qufkJs9EYlq8wbiyGZXL4UVVLTEnwesk0qTMdIpZiqOsknb9iyU3IWmEjnjPGm4wncXTbYr9I0PdJSg3u6LwqNzJbB8fCoCjo35qp2G9hyQAgHj7eP6uXt6RoUURD65DIaU%2F4EW67%2F999m177zsv24uc%2F7E%2BMkukhGBibg%2Fs8njiKjzcyC1dHKRc5SNXLMisQfUxKmerBCnB%2Bp50tBaHdE%2FXfre7%2BAQ%3D%3D&RelayState=49489 to

,artist,title,unified_song_id,date,release_date,this_day,this_week,days_since_release,day_of_week,rn,end_of_week,popularity,day_0,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,target,naive
0,Gucci Mane,Gingerbread Man,91914,2021-12-22,2009-12-04,952,6066.0,4401,5,1,False,23,952,850.0,811.0,750.0,834.0,967.0,918.0,821.0,976.0,1010.0,979.0,918.0,875.0,761.0,878.0,1022.0,948.0,935.0,907.0,877.0,768.0,5815,6039.0
1,Gucci Mane,Gingerbread Man,91914,2021-12-21,2009-12-04,850,6066.0,4400,4,1,False,23,850,811.0,750.0,834.0,967.0,918.0,821.0,976.0,1010.0,979.0,918.0,875.0,761.0,878.0,1022.0,948.0,935.0,907.0,877.0,768.0,951.0,5815,6099.0
2,Gucci Mane,Gingerbread Man,91914,2021-07-31,2009-12-04,878,6363.0,4257,1,1,False,23,878,1022.0,948.0,935.0,907.0,877.0,768.0,951.0,989.0,1061.0,991.0,964.0,886.0,859.0,949.0,1061.0,1025.0,1036.0,984.0,899.0,811.0,6066,6424.0
3,Gucci Mane,Gingerbread Man,91914,2021-07-30,2009-12-04,1022,6363.0,4256,0,1,False,23,1022,948.0,935.0,907.0,877.0,768.0,951.0,989.0,1061.0,991.0,964.0,886.0,859.0,949.0,1061.0,1025.0,1036.0,984.0,899.0,811.0,979.0,6066,6362.0
4,Gucci Mane,Gingerbread Man,91914,2021-07-29,2009-12-04,948,6771.0,4255,6,1,True,23,948,935.0,907.0,877.0,768.0,951.0,989.0,1061.0,991.0,964.0,886.0,859.0,949.0,1061.0,1025.0,1036.0,984.0,899.0,811.0,979.0,1054.0,6066,6587.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14567477,Tiesto & Lucas & Steve,ZENLESS,245214101,2024-07-06,2024-07-02,59943,61607.0,4,1,1,False,52,59943,74695.0,27206.0,15107.0,19281.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,159339,NaN
14567478,Tiesto & Lucas & Steve,ZENLESS,245214101,2024-07-05,2024-07-02,74695,61607.0,3,0,1,False,52,74695,27206.0,15107.0,19281.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,159339,NaN
14567479,Tiesto & Lucas & Steve,ZENLESS,245214101,2024-07-04,2024-07-02,27206,NaN,2,6,1,True,52,27206,15107.0,19281.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,159339,NaN
14567480,Tiesto & Lucas & Steve,ZENLESS,245214101,2024-07-03,2024-07-02,15107,NaN,1,5,1,False,52,15107,19281.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,195923,NaN


In [5]:
df2.to_csv('../csv_files/three_week_data.csv', index=False)